In [ ]:
import numpy as np
import verde as vd
import pandas as pd
import vtki
import PVGeo
import discretize

In [ ]:
topo = np.loadtxt('land-surface.topo', skiprows=1)
topo = vtki.PolyData(topo)
topo

In [ ]:
# Load detrended gravity data
_grav = np.loadtxt('grav-obs.grv', skiprows=1)
grav = vtki.PolyData(_grav[:, 0:3])
grav.point_arrays['residuals'] = _grav[:,3]
grav

In [ ]:
p = vtki.BackgroundPlotter()

In [ ]:
p.add_mesh(grav, name='gres')
p.add_mesh(topo, name='topo')
p.show_grid()

In [ ]:
# Create the TensorMesh
# Gravity inversion model space
def create_mesh(bounds, sz=100, pi=5, pad=None, extend=1000., zpad=None):
    if pad is None:
        pad = [10 * sz] * 3
    if zpad is None:
        zpad = [sz*2.5, sz*5, sz*10]
    xc = np.linspace(bounds[0]-extend, bounds[1]+extend, sz)
    xc = np.insert(xc, 0, np.min(xc) - np.cumsum(pad))
    xc = np.append(xc, np.max(xc) + np.cumsum(pad))

    yc = np.linspace(bounds[2]-extend, bounds[3]+extend, sz)
    yc = np.insert(yc, 0, np.min(yc) - np.cumsum(pad))
    yc = np.append(yc, np.max(yc) + np.cumsum(pad))
    
    zc = np.linspace(bounds[4], bounds[5], sz)
    zc = np.insert(zc, 0, np.min(zc) - np.cumsum(zpad))

    return vtki.RectilinearGrid(xc, yc, zc)
    

# Use topography bounds to constrain
bounds = topo.bounds
bounds[4] = -2100.0,
bounds[5] = topo.bounds[5] + 100
mesh = create_mesh(bounds, pad=[500, 500, 750, 1000, 1500, 2000])
p.add_mesh(mesh, show_edges=True, name='gm', opacity=0.7)
mesh

In [ ]:
dmesh, _ = discretize.TensorMesh.vtk_to_tensor_mesh(mesh)
dmesh.writeUBC('gravity-mesh.msh')

In [ ]:
# clip the gravity obeservation data
gc = grav.clip_box(mesh.bounds, invert=False)
data = pd.DataFrame(data=gc.points)
data['residuals'] = gc.point_arrays['residuals']
data['std'] = np.full(gc.n_points, 0.05)
with open('grav-obs-clipped.grv', 'w') as f:
    f.write('{}\n'.format(len(data)))
    data.to_csv(f, sep='\t', header=False,index=False)

In [ ]:
gc.plot()

In [ ]:
gc